In [ ]:
import torch
import numpy as np
from typing import List
import scipy
from scipy import linalg
from sklearn.decomposition import PCA
import pickle
import tqdm
torch.set_default_tensor_type(torch.FloatTensor)
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [ ]:
device = "cpu"

In [ ]:
q = torch.randn(10)

In [ ]:
with open("encodings.bert-base.250k.pickle", "rb") as f:
    data = pickle.load(f)
    
data = [d for d in data if d["gold"].lower() not in ["is", "are", "have", "had", "has","then","or","and",",",
                                                    ".","-","'","of","in","on","off","the","a","an", "be", "been", "from", "to",
                                                    "about", "around", "near", "it", "he", "she", "her","him","their", "they", "i", "was", "were","them",
                                                    "than"]]

In [ ]:
sents = np.array([d["sent"] for d in data])
masked_tokens = np.array([d["gold"] for d in data])
top_words = np.array([d["top_words"][0] for d in data])

In [6]:
print(sents[3])
# print(masked_tokens[2])
# print(top_words[2])

Omid released the instrumental album , Afterwords 3 , on Alpha Pup Records in 2007 .


In [7]:
H = np.array([d["vec_batch_norm"] for d in data])
with open("bert_embeddings.pickle", "rb") as f:
    W = pickle.load(f)
    
H,W = torch.from_numpy(H).to(device), torch.from_numpy(W).to(device)
H = H[:150000]
W = W.T
H.shape, W.shape


(torch.Size([150000, 768]), torch.Size([768, 30522]))

In [8]:
W.device, H.device

(device(type='cpu'), device(type='cpu'))

In [9]:
W.device

device(type='cpu')

In [10]:
# N = 100000
# d = 128
# l = 25
# H = (torch.randn(N,d) + torch.rand(N,d)**2)

# H = H - torch.mean(H, dim = 0, keepdim = True)
# H = H
# W = torch.randn(d,l)
cov_H = torch.tensor(np.cov(H.detach().cpu().numpy(), rowvar = False))

In [11]:
cov_H = cov_H.float()

In [12]:
del data

In [ ]:
# import time
# start = time.time()
# Q = cov_H[:1000]@W
# print(Q)
# print(cov_H.shape, W.shape)
# print(time.time() - start)

In [ ]:
# u = torch.nn.Parameter(u)
# optimizer = torch.optim.SGD([u], lr=0.001, momentum=0.9, weight_decay=1e-4)

In [13]:
def get_cov_output_projected(u, cov_H, W):
    
    u_normed = u / torch.norm(u)
    P = torch.eye(cov_H.shape[0]) - (u_normed@u_normed.T)
    #P = u_normed@u_normed.T
    print(P.shape, W.shape, cov_H.shape, u_normed.shape)
    first = P@W
    print("done first")
    second = cov_H@first
    print("done second")
    third = P@second
    print("done third")
    fourth = W.T@third
    print("done fourth")
    return fourth
    #return W.T@P@cov_H@P@W

def get_cov_output_total(H,W,n=10000):
    with torch.no_grad():
        Y_hat = H[:n]@W 
        Y_hat = Y_hat - torch.mean(Y_hat, dim = 1, keepdim = True)
        return torch.sum(Y_hat*Y_hat)/Y_hat.shape[0]
    #return torch.tensor(np.cov(Y_hat.detach().cpu().numpy(), rowvar = False))

def eval_total_var(H,W):
    
    with torch.no_grad():
        k = 1000
        Y_sum = torch.zeros(W.shape[1]).to(device)
        Y_sqr_sum = torch.zeros(W.shape[1]).to(device)
    
        for i in range(0,len(H), k):
            Y = H[i:i+k]@W
            Y_sum += torch.sum(Y, dim = 0)
            Y_sqr_sum += torch.sum(Y**2, dim = 0)
    
        Y_sqr_sum /= len(H)
        Y_sum /= len(H)
        return (Y_sqr_sum - Y_sum**2).mean().detach().cpu().numpy().item() # mean over vocab
    
def get_loss_func(cov_output_projected):
    
    loss =  torch.sum(torch.diag(cov_output_projected))
    print("done loss calculation")
    return loss

def get_loss_func2(u, cov_H, W):

    u_normed = u / torch.norm(u)
    P = torch.eye(cov_H.shape[0]) - (u_normed@u_normed.T)
    #print(P.dtype, W.dtype, cov_H.dtype)
    #P = u_normed@u_normed.T
    first = P@W
    second = cov_H@first
    third = P@second
    fourth = torch.sum(W*third)/third.shape[1] # mean over vocab
    return fourth 

def get_projection_to_intersection_of_nullspaces(rowspace_projection_matrices: List[np.ndarray], input_dim: int):
    """
    Given a list of rowspace projection matrices P_R(w_1), ..., P_R(w_n),
    this function calculates the projection to the intersection of all nullspasces of the matrices w_1, ..., w_n.
    uses the intersection-projection formula of Ben-Israel 2013 http://benisrael.net/BEN-ISRAEL-NOV-30-13.pdf:
    N(w1)∩ N(w2) ∩ ... ∩ N(wn) = N(P_R(w1) + P_R(w2) + ... + P_R(wn))
    :param rowspace_projection_matrices: List[np.array], a list of rowspace projections
    :param dim: input dim
    """

    I = np.eye(input_dim)
    Q = np.sum(rowspace_projection_matrices, axis = 0)
    P = I - get_rowspace_projection(Q)

    return P

def get_rowspace_projection(W: np.ndarray) -> np.ndarray:
    """
    :param W: the matrix over its nullspace to project
    :return: the projection matrix over the rowspace
    """

    if np.allclose(W, 0):
        w_basis = np.zeros_like(W.T)
    else:
        w_basis = scipy.linalg.orth(W.T) # orthogonal basis

    P_W = w_basis.dot(w_basis.T) # orthogonal projection on W's rowspace

    return P_W

def get_first_pca(H):
    pca = PCA(n_components = 1)
    pca.fit(H)
    return torch.from_numpy(pca.components_.T)

def BCA(H,W,n_components, eps = 1e-8, max_iters = 1500, min_iters = 200, init_pca = True):
    
    P_nullspace = torch.eye(H.shape[1])
    results = []
    #cov_out_total = get_cov_output_total(H,W)
    total_var_orig = eval_total_var(H,W) #get_loss_func(cov_out_total).detach().cpu().numpy().item()
    remaining_var = total_var_orig
    print("Total var original: ", remaining_var)
    H_proj = H.clone()
    rowspace_projs = []
    
    for i in range(n_components):
        print("****" + "ITER {}".format(i) + "****")
        H_proj = H@P_nullspace # remove previous component 
        cov_H = torch.from_numpy(np.cov(H_proj.detach().cpu().numpy(), rowvar = False)).float()
        
        if init_pca:
            u = get_first_pca(H_proj.detach().cpu().numpy())
        else:
            u = 1e-2*torch.randn(H_proj.shape[1], 1)
        u = torch.nn.Parameter(u)
        #optimizer = torch.optim.SGD([u], lr=1)
        optimizer = torch.optim.Adam([u])
        
        diff = 10
        j = 0
        loss_vals = [np.inf]
        patience = 10
        patience_counter = 0
        
        while j < max_iters and patience_counter < patience:
            optimizer.zero_grad()
            #cov_out = get_cov_output_projected(u,cov_H,W)
            #loss = get_loss_func(cov_out)
            loss = get_loss_func2(u, cov_H, W)
            loss.backward()
            optimizer.step()
            loss_vals.append(loss.detach().cpu().numpy().item())
            diff = np.abs(loss_vals[-1] - loss_vals[-2])
            
            if diff > eps:
                patience_counter = 0
            else:
                if j > min_iters:
                    patience_counter += 1
                
            if j % 50 == 0: print("j, loss, ", j, loss.detach().cpu().numpy().item(), diff)
            j += 1
        print("finished after {} iters".format(j))
        
        # calculate new nullspace projection to neutralzie component u
        
        u_normed = u / torch.norm(u)
        rowspace_projs.append((u_normed@u_normed.T).detach().cpu().numpy())
        P_nullspace = torch.from_numpy(get_projection_to_intersection_of_nullspaces(rowspace_projs,cov_H.shape[0])).float()
        #P_nullspace = torch.eye(H_proj.shape[1]).double() - u_normed@u_normed.T
        
        # calcualte explained variance
        #cov_out_total = get_cov_output_total(H,W)
        total_var = eval_total_var(H,W)
        #cov_out_projected = get_cov_output_projected(u,cov_H,W)
        #total_var_projected = get_loss_func(cov_out_projected).detach().cpu().numpy().item()
        total_var_projected = remaining_var - get_loss_func2(u,cov_H,W).detach().cpu().numpy().item()
        explained_var = total_var_projected / total_var_orig
        remaining_var = remaining_var - total_var_projected
        
        u = u / u.norm()
        results.append({"vec": u.squeeze().detach().cpu().numpy(), "projected_var": total_var_projected,
                       "total_var": total_var, "explained_var": explained_var*100})
        
        with open("bca.pickle", "wb") as f:
            pickle.dump(results, f)
            
    return results
        

In [14]:
bca = BCA(H,W,n_components=1, eps = 0.5*1e-4, init_pca = True)

Total var original:  6.112844467163086
****ITER 0****
j, loss,  0 5.748241424560547 inf
j, loss,  50 4.944725513458252 0.01856088638305664
j, loss,  100 4.428926467895508 0.004486083984375
j, loss,  150 4.293493270874023 0.0017309188842773438
j, loss,  200 4.229542255401611 0.0009198188781738281
j, loss,  250 4.197164535522461 0.000438690185546875
j, loss,  300 4.181656837463379 0.00021791458129882812
j, loss,  350 4.173521041870117 0.00012159347534179688
j, loss,  400 4.168801784515381 7.486343383789062e-05
j, loss,  450 4.165884971618652 4.5299530029296875e-05
finished after 453 iters


In [ ]:
with open("bca.pickle", "wb") as f:
    pickle.dump(bca, f)

In [ ]:
with open("bca.pickle", "rb") as f:
    bca = pickle.load(f)

In [15]:
vecs = np.array([x["vec"] for x in bca])
for i,v in enumerate(vecs):
    for j, v2 in enumerate(vecs):
        if j <= i: continue
            
        print(i,j, v@v2.T)

In [ ]:
import scipy.optimize as optimize

def f(u,W,V, orthogonal_constraints = []):
    norm = np.linalg.norm
    grad =  0.5 * ( (norm(u)**2/norm(V.T@u)**2)*V@V.T@u  + (norm(u)**2/norm(W.T@u)**2)*W@W.T@u)
    c_orth = 1
    for v in orthogonal_constraints:
        grad += -c_orth*(1/(u.T@v)**2)*v
        
    return grad

def f2(u,W,V,cov):
    
    S = cov
    t_0 = (u).dot(u)
    t_1 = (t_0 ** 4)
    t_2 = (W).dot((W.T).dot(u))
    t_3 = (u).dot(t_2)
    t_4 = (S).dot(u)
    t_5 = (1 / t_1)
    t_6 = (u).dot(t_4)
    t_7 = t_4
    t_8 = (4 / (t_0 ** 5))
    t_9 = (u).dot(t_7)
    t_10 = (t_3 * u)
    functionValue = ((t_6 * t_3) / t_1)
    gradient = ((((((t_5 * t_3) * t_4) + (t_5 * (t_3 * t_7))) + ((t_5 * t_6) * t_2)) + (t_5 * (t_9 * t_2))) - (((t_8 * t_6) * t_10) + ((t_8 * t_9) * t_10)))
    return gradient

def calculate_components(H,cov_H,W,n=2):
    comps = []
    H_proj = H.detach().cpu().numpy()
    cov = cov_H.detach().cpu().numpy()
    V = np.linalg.cholesky(cov)
    W = W.detach().cpu().numpy()
    P_nullspace = np.eye(768)
    
    for i in range(n):
        print("Iteration {}".format(i))
        print("=========================")
        u = 0.1*np.random.randn(768)
        u_last = u.copy()
        
        for j in range(100):
            u_last, u = u, f(u,W,V,comps)
            diff = np.linalg.norm(u_last-u)
            if diff < 1e-5: break
            if j%10 == 0: print(diff)
        
        comps.append(u/np.linalg.norm(u))

        print("Start test.")
        
        for k in range(len(comps)):
            for t in range(len(comps)):
                if k <= t: continue
                print("Test", comps[k].T@comps[t])
                
        print("Done test.")
        
        #P_nullspace = np.eye(768) - u@u.T
        #H_proj = H_proj@P_nullspace
        #cov = np.cov(H_proj, rowvar = False)
        #V = np.linalg.cholesky(cov)
    
    return comps 

comps = calculate_components(H,cov_H,W,n=2)

# u = np.random.randn(768,1)
# u_last = u.copy()

# i = 0
# while i < 100:
#     u_last, u = u, f(u)
#     print(np.linalg.norm(u_last-u))
#     i+=1

Iteration 0
14.293095243475268
1.8796024192443417
0.19602017222066867
0.03468219526766493
0.007907824977824478
0.0018865601779827132
0.0004523839528763313
0.00010860775992305265


In [31]:
comps[0].T@comps[1]

-0.9999999999993907

In [ ]:

#for i in range(len(bca)):
    
    #print(bca[i]["explained_var"])
    
#cov_out_total = get_cov_output_total(H,W)
#total_var = get_loss_func(cov_out_total).detach().cpu().numpy().item()    
vars = [x["explained_var"] for x in bca]
print(vars)
print(sum(vars))

In [22]:
u = comps[0] #bca[0]["vec"]
#u = np.expand_dims(u, axis=1)
vecs = H.detach().cpu().numpy()
vecs_u = vecs@u
idx = np.argsort(vecs_u)
k = 250
top_neg, top_pos = idx[:k], idx[-k:]

sents_pos = sents[top_pos]
sents_neg = sents[top_neg]
gold_pos = masked_tokens[top_pos]
gold_neg = masked_tokens[top_neg]
preds_pos = top_words[top_pos]
preds_neg = top_words[top_neg]

for i in range(30):
    print(gold_pos[i], " -------- ", sents_pos[i], "---------", vecs_u[idx[i]])
    print("======================================")
    
print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&")

for i in range(10):
    print(gold_neg[i], " -------- ", sents_neg[i])
    print("======================================")

aspects  --------  There are also many social and political aspects that affect this system of marriage , throughout different societies and species . --------- -16.814509578906517
provisions  --------  These provisions , however , only apply to public bills that originate in the House of Commons . --------- -16.602402157293543
diaries  --------  Many individuals and some companies and groups use web logs or blogs , which are largely used as easily updatable online diaries . --------- -16.24923505438625
enemies  --------  Susan had hidden knowledge of the child from Dresden for the child 's safety not just because of his list of enemies , but hers as well . --------- -16.073949970223463
theory  --------  Hoxhaists also argue that the theory of New Democracy and People 's War were revisionist and anti-scientific . --------- -16.05632798176108
openings  --------  Several small openings were made in the structure for the positioning of bows and arrows , which can be seen even today . ----

In [ ]:
idx = np.argsort(vecs_u)
vecs_u.shape

In [ ]:
np.linalg.norm(u)

In [ ]:
bca[-1]["cov_out"]

In [ ]:
bca[-3]["explained_var"]

In [ ]:
u,v = np.random.randn(768), np.random.randn(768).T
u,v = u / np.linalg.norm(u), v/np.linalg.norm(v)
u@v

In [ ]:
a = [1,2,3,4,5,6,7,8,9,10,11,12]

In [ ]:
a[-12:-2]

In [ ]:
A = np.random.rand(768,32000)
H = np.random.rand(10000,768)
H*A

In [ ]:
W = torch.randn(768,32000)
H = torch.randn(10000, 768)
Y_hat = H[:10000]@W 
torch.mean(Y_hat*Y_hat)

In [ ]:
H.shape, W.shape

In [ ]:
H_first_comp = get_first_pca(H)
Y = H[:10000]@W
Y_first_comp = get_first_pca(Y)

In [ ]:
total_var = eval_total_var(H,W)
print(total_var)

In [ ]:
print(total_var.sum()/W.shape[1])

In [ ]:
u = torch.randn(768,1)
get_loss_func2(H_first_comp, cov_H, W)

In [ ]:
5%1